In [ ]:
 ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

     / 38.3MB 6.5MB/s
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 1.1MB 9.0MB/s 
     |████████████████████████████████| 3.2MB 18.6MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 81kB 260kB/s 
     |████████████████████████████████| 296kB 46.1MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.12.0-py2.py3-none-any.whl size=243829 sha256=fe8d874a9680595e71a3eedf306e67005c95a457e6e82346f6d71e9ca92bc0b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wjkvx0mg/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=1f0a1f330aaa47b2b56063a89e11450307fb194b239f8b0a0e7610a6ea6807a8
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=b

In [259]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from math import ceil
import time


sns.set()
import warnings
warnings.filterwarnings('ignore')

# Read Data

In [260]:
url = 'https://github.com/joaopfonseca/business-cases/blob/7d95f4d49e04d2eabeae742310eecd30b419251e/BC2_predicting_cancellations/H2.csv?raw=true'
df = pd.read_csv(url)

In [261]:
df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79330 entries, 0 to 79329
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IsCanceled                   79330 non-null  int64  
 1   LeadTime                     79330 non-null  int64  
 2   ArrivalDateYear              79330 non-null  int64  
 3   ArrivalDateMonth             79330 non-null  object 
 4   ArrivalDateWeekNumber        79330 non-null  int64  
 5   ArrivalDateDayOfMonth        79330 non-null  int64  
 6   StaysInWeekendNights         79330 non-null  int64  
 7   StaysInWeekNights            79330 non-null  int64  
 8   Adults                       79330 non-null  int64  
 9   Children                     79326 non-null  float64
 10  Babies                       79330 non-null  int64  
 11  Meal                         79330 non-null  object 
 12  Country                      79306 non-null  object 
 13  MarketSegment   

In [263]:
df.describe(include='all')

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
count,79330.000000,79330.000000,79330.000000,79330,79330.000000,79330.000000,79330.000000,79330.000000,79330.000000,79326.000000,79330.000000,79330,79306,79330,79330,79330.000000,79330.000000,79330.000000,79330,79330,79330.000000,79330,79330,79330,79330.000000,79330,79330.000000,79330.000000,79330.000000,79330,79330
unique,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,166,8,5,NaN,NaN,NaN,8,9,NaN,3,224,208,NaN,4,NaN,NaN,NaN,3,864
top,NaN,NaN,NaN,August,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BB,PRT,Online TA,TA/TO,NaN,NaN,NaN,A,A,NaN,No Deposit,9,NULL,NaN,Transient,NaN,NaN,NaN,Check-Out,2015-10-21
freq,NaN,NaN,NaN,8983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62305,30960,38748,68945,NaN,NaN,NaN,62595,57007,NaN,66442,31955,75641,NaN,59404,NaN,NaN,NaN,46228,1416
mean,0.417270,109.735724,2016.174285,NaN,27.177449,15.786625,0.795185,2.182957,1.850977,0.091370,0.004941,NaN,NaN,NaN,NaN,0.025615,0.079743,0.132371,NaN,NaN,0.187369,NaN,NaN,NaN,3.226774,NaN,105.304465,0.024367,0.546918,NaN,NaN
std,0.493111,110.948526,0.699181,NaN,13.398523,8.728451,0.885026,1.456416,0.509292,0.372177,0.084323,NaN,NaN,NaN,NaN,0.157983,0.415472,1.693411,NaN,NaN,0.608620,NaN,NaN,NaN,20.870890,NaN,43.602954,0.154919,0.780776,NaN,NaN
min,0.000000,0.000000,2015.000000,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN
25%,0.000000,23.000000,2016.000000,NaN,17.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,79.200000,0.000000,0.000000,NaN,NaN
50%,0.000000,74.000000,2016.000000,NaN,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,99.900000,0.000000,0.000000,NaN,NaN
75%,1.000000,163.000000,2017.000000,NaN,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,126.000000,0.000000,1.000000,NaN,NaN


# Data Understanding

In [264]:
df.duplicated().sum()
#these observations were not considered as duplicates, but as similar bookings of different people

25902

In [265]:
df.isna().sum()

IsCanceled                      0
LeadTime                        0
ArrivalDateYear                 0
ArrivalDateMonth                0
ArrivalDateWeekNumber           0
ArrivalDateDayOfMonth           0
StaysInWeekendNights            0
StaysInWeekNights               0
Adults                          0
Children                        4
Babies                          0
Meal                            0
Country                        24
MarketSegment                   0
DistributionChannel             0
IsRepeatedGuest                 0
PreviousCancellations           0
PreviousBookingsNotCanceled     0
ReservedRoomType                0
AssignedRoomType                0
BookingChanges                  0
DepositType                     0
Agent                           0
Company                         0
DaysInWaitingList               0
CustomerType                    0
ADR                             0
RequiredCarParkingSpaces        0
TotalOfSpecialRequests          0
ReservationSta

## Pandas Profilling

In [266]:
#profile=ProfileReport(df)

In [267]:
#profile.to_notebook_iframe()

# Data Preparation

In [268]:
#replace NA with 0
df.replace(np.nan,0, inplace=True)

## Data cleaning

In [269]:
#take out spaces of 'null'
df['Company']=df['Company'].apply(lambda x : x.lstrip()) 
df['Agent']=df['Agent'].apply(lambda x : x.lstrip())

In [270]:
df.replace('NULL',0, inplace=True)

In [271]:
#taking out adults = 0 and children =0 as we considered them as mistakes.
#reseting index
list_drop = df[(df['Children'] ==0)& (df['Adults'] == 0)].index.to_list()
df.drop(list_drop,axis=0,inplace=True)
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,6,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,0,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,1,Canceled,2015-04-02


In [272]:
list_drop = df[(df['Children'] ==0)& (df['Adults'] == 0)].index.to_list()
len(list_drop)

0

## Feature Engineering

In [273]:
df["NumberOfPreviousBookings"]=df["PreviousCancellations"]+df["PreviousBookingsNotCanceled"]

In [274]:
df['RationPreviousCancelled']=(df["PreviousCancellations"]/(df["PreviousBookingsNotCanceled"]+df["PreviousCancellations"])).replace(np.inf, 0)
df['RationPreviousCancelled']= df['RationPreviousCancelled'].replace(np.nan, 0)

In [275]:
#create Check In Feature
#create month dictionary
month = {	'Janauary':'01','February':'02','March':'03','April':'04','May':'05','June':'06','July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'	}
#change to pandas df
month_df = pd.DataFrame.from_dict(month, orient='index', columns=['month_no'])
#merge df with months
df = df.merge(month_df, left_on=df['ArrivalDateMonth'], right_index=True)
#create date variable
df['CheckInDate'] = pd.to_datetime(dict(year=df.ArrivalDateYear, month=df.month_no, day=df.ArrivalDateDayOfMonth))


In [276]:
#cosin of Day of Year
df['DayOfYear'] = df['CheckInDate'].dt.dayofyear
df['DayOfYear']=np.cos(df['DayOfYear'])

In [277]:
#features deleted from list_of_features & categorical_features:  'ArrivalDateDayOfMonth', 'DepositType'

In [278]:
list_of_features=['LeadTime', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber','StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType',
       'BookingChanges', 'Agent', 'Company','AssignedRoomType',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests','NumberOfPreviousBookings','DayOfYear','IsCanceled','RationPreviousCancelled']
categorical_features=['ArrivalDateMonth',"ArrivalDateWeekNumber",
                      'Meal','MarketSegment','DistributionChannel',"IsRepeatedGuest",
                      'ReservedRoomType',"Agent",'AssignedRoomType',
                      "Company","CustomerType"]

# One Hot Encoding

In [279]:
for i in categorical_features:
    df[i]=df[i].astype(str)

In [280]:
df_train = df.copy()
# Use OneHotEncoder to encode the categorical features. Get feature names and create a DataFrame 
# with the one-hot encoded categorical features 
#df_train.drop(columns=["Agent","Company"],inplace=True)
ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(df_train[categorical_features])
ohc_feat_names = ohc.get_feature_names()
df_train = pd.DataFrame(ohc_feat, index=df_train.index, columns=ohc_feat_names)
df=df[list_of_features]
encoded = pd.concat([df.drop(columns=categorical_features), df_train], axis=1)

In [281]:
data = encoded

In [282]:
data

,LeadTime,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,PreviousCancellations,PreviousBookingsNotCanceled,BookingChanges,DaysInWaitingList,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,NumberOfPreviousBookings,DayOfYear,IsCanceled,RationPreviousCancelled,x0_August,x0_December,x0_February,x0_July,x0_June,x0_March,x0_May,x0_November,x0_October,x0_September,x1_11,x1_12,x1_13,x1_14,x1_15,x1_16,x1_17,x1_18,x1_19,x1_20,x1_21,x1_22,x1_23,...,x9_452,x9_46,x9_465,x9_466,x9_47,x9_477,x9_478,x9_479,x9_48,x9_481,x9_483,x9_485,x9_486,x9_489,x9_49,x9_491,x9_492,x9_494,x9_497,x9_51,x9_62,x9_65,x9_67,x9_68,x9_71,x9_72,x9_73,x9_76,x9_78,x9_8,x9_81,x9_84,x9_85,x9_9,x9_91,x9_93,x9_96,x10_Group,x10_Transient,x10_Transient-Party
0,6,0,2,1,0.0,0,0,0,0,0,0.00,0,0,0,0.977533,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,88,0,4,2,0.0,0,0,0,0,0,76.50,0,1,0,0.977533,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,65,0,4,1,0.0,0,0,0,0,0,68.00,0,1,0,0.977533,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,92,2,4,2,0.0,0,0,0,0,0,76.50,0,2,0,0.977533,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,100,0,2,2,0.0,0,0,0,0,0,76.50,0,1,0,0.705530,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75225,43,2,5,3,0.0,0,0,0,0,0,154.93,0,0,0,-0.598460,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
75231,245,2,5,2,0.0,0,0,0,0,0,91.93,0,1,0,-0.598460,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
75238,175,2,5,2,0.0,0,0,0,0,0,91.93,0,0,0,-0.598460,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
75309,184,2,5,2,0.0,0,0,0,0,0,108.51,0,2,0,0.350797,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Splitting the Data

In [283]:
X = data.drop('IsCanceled',axis=1)
y= data['IsCanceled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

# Outliers

In [284]:
numerical_features=['LeadTime', 'StaysInWeekendNights', 'StaysInWeekNights','PreviousCancellations','PreviousBookingsNotCanceled', 'BookingChanges','DaysInWaitingList', 'ADR',
       'TotalOfSpecialRequests','NumberOfPreviousBookings']

In [285]:
sns.set()
fig, axes = plt.subplots(2, ceil(len(numerical_features) / 2), figsize=(27, 10))
for ax, feat in zip(axes.flatten(), numerical_features):
  sns.boxplot(X_train[feat], ax=ax)
title = "Numeric Variables"
plt.suptitle(title)
plt.show()

In [286]:
#outlier removal
df_new = X_train[~(X_train['LeadTime'] >= 550)]  
df_new = df_new[~(df_new['StaysInWeekendNights'] >= 10)]  
df_new = df_new[~(df_new['ADR'] >= 1000)]  

X_train = df_new

In [287]:
#BOXPLOTS after Outlier removal
sns.set()
fig, axes = plt.subplots(2, ceil(len(numerical_features) / 2), figsize=(27, 10))
for ax, feat in zip(axes.flatten(), numerical_features):
  sns.boxplot(X_train[feat], ax=ax)
title = "Numeric Variables"
plt.suptitle(title)
plt.show()

In [288]:
#y train delete outliers rows
merged_Train = X_train.merge(y_train, how='inner', right_index=True, left_index = True)
y_train = merged_Train['IsCanceled']

# Feature Selection

In [289]:
dt = DecisionTreeClassifier(random_state=15,max_depth = 12,criterion='entropy')
dt.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=15, splitter='best')

In [290]:
fi_col = []
fi = []
for i,column in enumerate(X_train):
    print('The feature importance for {} is : {}'.format(column,dt.feature_importances_[i]))
    fi_col.append(column)
    fi.append(dt.feature_importances_[i])

The feature importance for LeadTime is : 0.16078588840072425
The feature importance for StaysInWeekendNights is : 0.0027119092128993104
The feature importance for StaysInWeekNights is : 0.008823704734500578
The feature importance for Adults is : 0.005243962458728284
The feature importance for Children is : 0.0014140111133101852
The feature importance for Babies is : 0.0
The feature importance for PreviousCancellations is : 0.0
The feature importance for PreviousBookingsNotCanceled is : 0.0
The feature importance for BookingChanges is : 0.01965366939701888
The feature importance for DaysInWaitingList is : 0.010458195917970177
The feature importance for ADR is : 0.028698138656002396
The feature importance for RequiredCarParkingSpaces is : 0.02246384068509871
The feature importance for TotalOfSpecialRequests is : 0.18889765719053714
The feature importance for NumberOfPreviousBookings is : 0.00038809089808849496
The feature importance for DayOfYear is : 0.013003887386782024
The feature imp

In [291]:
fi_df = zip(fi_col,fi)
fi_df = pd.DataFrame(fi_df,columns = ['Features','Features Importance'])
fi_df = fi_df.sort_values('Features Importance',ascending= False).reset_index().drop('index',axis=1)
fi_df

,Features,Features Importance
0,TotalOfSpecialRequests,0.188898
1,LeadTime,0.160786
2,RationPreviousCancelled,0.140122
3,x10_Transient,0.106753
4,x3_Groups,0.072487
...,...,...
513,x7_354,0.000000
514,x7_35,0.000000
515,x7_346,0.000000
516,x7_344,0.000000


In [292]:
fi_df15 = fi_df[0:10]
sns.set_style('whitegrid')
sns.barplot(fi_df15['Features Importance'], fi_df15['Features'], palette = 'cubehelix', orient = 'h') 

In [293]:
fi_df = fi_df[fi_df['Features Importance'] > 0]
features_selected= fi_df['Features'].tolist()

In [294]:
len(features_selected)

107

In [295]:
X_train = X_train[features_selected]
X_test = X_test[features_selected]

In [296]:
X_val = X_val[features_selected]

# Functions/K-FOLD

In [298]:
X=pd.concat([X_train,X_test]).reset_index(drop=True)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
y=pd.concat([y_train,y_test]).reset_index(drop=True)

In [301]:
def avg_score(model):
    # apply kfold
    kf = KFold(n_splits=10)
    # create lists to store the results from the different models 
    score_train = []
    score_test = []
    timer = []
    for train_index, test_index in kf.split(X):
        # get the indexes of the observations assigned for each partition
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # start counting time
        begin = time.perf_counter()
        # fit the model to the data
        model.fit(X_train, y_train)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = f1_score(y_train,model.predict(X_train))
        # check the mean accuracy for the test
        value_test = f1_score(y_test,model.predict(X_test))
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_test = round(np.mean(score_test),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_test = round(np.std(score_test),2)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test)

In [302]:
def show_results(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test = avg_score(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test
        count+=1
    return df

# Random Forest

In [303]:
modelRF = RandomForestClassifier(random_state = 10,criterion='gini').fit(X_train,y_train)

In [304]:
predictionsRF1 = modelRF.predict(X_test)

In [305]:
cm = confusion_matrix(y_test,predictionsRF1)
cm_norm = cm/cm.sum(axis=1).reshape(-1,1)
plot_confusion_matrix(modelRF,X_test,y_test)
plt.show()

In [306]:
confusion_matrix(y_test,predictionsRF1)

array([[8107,  666],
       [1366, 4948]])

In [307]:
round(f1_score(y_test,modelRF.predict(X_test)),2)

0.83

In [ ]:
modelRF.score(X_test, y_test)

0.8653145091800888

In [ ]:
  print(classification_report(y_test, predictionsRF1))

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      8773
           1       0.88      0.78      0.83      6314

    accuracy                           0.87     15087
   macro avg       0.87      0.85      0.86     15087
weighted avg       0.87      0.87      0.86     15087



In [ ]:
fig = plt.figure(figsize = (10,6))
importances = modelRF.feature_importances_
features = X_train.columns
data_tuples = list(zip(importances, features))
data = pd.DataFrame(data_tuples, columns = ['importances', 'features'])
data = data.sort_values('importances', ascending = False)
data = data[0:5]
sns.barplot(data['importances'], data['features'], palette = 'cubehelix', orient = 'h')

In [ ]:
rf = pd.DataFrame(columns = ['Time','Train','Test'],index=['Random Forest'])
show_results(rf,modelRF)

,Time,Train,Test
Random Forest,15.974+/-0.1,0.99+/-0.0,0.835+/-0.0


In [ ]:
#validation

In [308]:
predictionsRF2 = modelRF.predict(X_val)

In [309]:
cm = confusion_matrix(y_val,predictionsRF2)
cm_norm = cm/cm.sum(axis=1).reshape(-1,1)
plot_confusion_matrix(modelRF,X_val,y_val)
plt.show()

In [310]:
confusion_matrix(y_val,predictionsRF2)

array([[6466,  510],
       [1073, 4021]])

In [311]:
round(f1_score(y_val,modelRF.predict(X_val)),2)

0.84

In [312]:
modelRF.score(X_val, y_val)

0.8688483844241922

# Monte Carlo Simulation

In [1]:
begin=time.perf_counter()
samples=100
loops=100000
percentage_cancelation=[]
for i in range(loops):
  dataset_sample=X_test.sample(n=samples)
  percentage=sum([x[1] for x in modelRF.predict_proba(dataset_sample)])/samples
  percentage_cancelation.append(percentage)
end=time.perf_counter()
end-begin

NameError: name 'time' is not defined

In [ ]:
ax=sns.displot(percentage_cancelation["prob"], kde=True,stat="probability")
ax.set(xlabel='Percentage of cancellations', ylabel='Probability Density')
plt.show()

In [ ]:
np.quantile(percentage_cancelation["prob"],0.05)